In [1]:
from hdx.utilities.easy_logging import setup_logging
from hdx.hdx_configuration import Configuration
from hdx.data.dataset import Dataset
import numpy as np
import pandas as pd

In [2]:
setup_logging()

No logging configuration parameter. Using default.
Loading logging configuration from: c:\users\vanis\anaconda3\lib\site-packages\hdx\utilities\logging_configuration.yml


In [3]:
Configuration.create(hdx_site='prod', user_agent='A_Quick_Example')

INFO - 2019-02-20 16:45:14 - hdx.hdx_configuration - No HDX base configuration parameter. Using default base configuration file: c:\users\vanis\anaconda3\lib\site-packages\hdx\hdx_base_configuration.yml.
INFO - 2019-02-20 16:45:14 - hdx.hdx_configuration - Loading HDX base configuration from: c:\users\vanis\anaconda3\lib\site-packages\hdx\hdx_base_configuration.yml
INFO - 2019-02-20 16:45:14 - hdx.hdx_configuration - No HDX configuration parameter and no configuration file at default path: C:\Users\vanis\.hdx_configuration.yml.
INFO - 2019-02-20 16:45:14 - hdx.hdx_configuration - Read only access to HDX: False


'https://data.humdata.org/'

In [4]:
from hdx.facades.simple import facade

No logging configuration parameter. Using default.
Loading logging configuration from: c:\users\vanis\anaconda3\lib\site-packages\hdx\utilities\logging_configuration.yml


In [5]:
datasets = Dataset.get_all_dataset_names()
print(len(datasets)) #Total number of datasets obtained using the API


12136


In [6]:
#Script to clean the datasets list and return the list of datasets that have at least 1
#resource of the required file type(s).
#Currently very naive and inefficient
def clean(datasets, file_types=[]):
    result = []
    index = 0
    while (index < len(datasets)):
        temp_dataset = Dataset.read_from_hdx(datasets[index])
        temp_dataset.separate_resources() #In order to use dataset.resources
        if (len(temp_dataset.resources) > 0):
            if (len(file_types) > 0):
                if (not set(temp_dataset.get_filetypes()).isdisjoint(file_types)): #To check if elements of the 2 lists of file_types overlap.
                    result.append(datasets[index])
            else :
                result.append(datasets[index])
        index += 1
    return result

In [36]:
def dload_dset(dataset_1,dload_path,row_limit=10,file_type='CSV') :
    if (file_type == None):
        url, path = dataset_1.resources[0].download(dload_path)
        pandas_dataset = pd.read_csv(path)
    else :
        if (file_type not in dataset_1.get_filetypes()):
            url = '1'
            path = '2'
            pandas_dataset ='9'
            return 'Error: Required file type not in dataset OR dataset does not contain any resources.'
        else :
            url, path = dataset_1.resources[dataset_1.get_filetypes().index(file_type)].download(dload_path)
            pandas_dataset = pd.read_csv(path, encoding='latin-1')
    pandas_dataset = pandas_dataset.head(row_limit)
    return pandas_dataset

In [39]:
#hdx_dset_scraper downloads a certain number (limit) of datasets from HDX to the given 
#download path of the given file_type (currently only 'CSV' is supported due to pd.read_csv). To use, 
#import both hdx_dset_scraper and dload_dset.
def hdx_dset_scraper(dload_path,limit=1000000000,download=True,file_type='CSV'):
    datasets = Dataset.get_all_dataset_names()
    if (limit > len(datasets)):
        limit = len(datasets)
    dset_dict = {}
    index = 0
    while (limit > index) :
        dataset_1 = Dataset.read_from_hdx(datasets[index]) 
        dataset_1.separate_resources()
        if (file_type not in dataset_1.get_filetypes()) :
            index += 1
            limit += 1
        else :
            dset_dict.update({datasets[index] : dload_dset(dataset_1,dload_path)})
            index += 1
    return dset_dict

In [40]:
dict = hdx_dset_scraper('Datasets',limit=10)
print(dict)

{'160516-ecuador-earthquake-4w-1st-round':                                         Organizacion                 Sector  \
0  Alto Comisionado de las Naciones Unidas para l...  Recuperación Temprana   
1                           Catholic Relief Services  Recuperación Temprana   
2                           Catholic Relief Services  Recuperación Temprana   
3                           Catholic Relief Services  Recuperación Temprana   
4                           Catholic Relief Services  Recuperación Temprana   
5                                        ONU Habitat  Recuperación Temprana   
6                                        ONU Habitat  Recuperación Temprana   
7                                        ONU Habitat  Recuperación Temprana   
8                                        ONU Mujeres  Recuperación Temprana   
9                                        ONU Mujeres  Recuperación Temprana   

   Pcode    Provincia Canton Total  
0    NaN  Desconocido    NaN   NaN  
1    NaN  Des

9                     ACF  }


,objectid,event_,county_,district_,division_,date_,place,deaths_,injured_,missing_,...,other_responders_,house__iron_sheet_,house__wooden_,house__mud_,house__grass_thatched_,house__tent_,x,y,constituency_gis,county_centriod
0,1001,FLOOD,WAJIR,Wajir North,WAJIR-BOR,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.006965,40.535670,WAJIR WEST,"(1.80813762380, 40.03498563960)"
1,1002,DROUGHT,MANDERA,MANDERA,WARGADUD,NaN,Wargadud village,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.068411,40.636267,MANDERA SOUTH,"(3.43646948803, 40.73881016670)"
2,1003,FOREST FIRE,NAIROBI,NAIROBI,WESTLANDS,NaN,Karura forest,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1.241644,36.783359,KAMUKUNJI,"(-1.29341347028, 36.86818926820)"
3,1004,DROUGHT,NAIROBI,NAIROBI,WESTLANDS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1.241644,36.783359,KAMUKUNJI,"(-1.29341347028, 36.86818926820)"
4,1005,FLOOD,NAIROBI,NAIROBI,WESTLANDS,NaN,Kitisuru,NaN,NaN,NaN,...,GOK,NaN,NaN,NaN,NaN,NaN,-1.241644,36.783359,KAMUKUNJI,"(-1.29341347028, 36.86818926820)"
5,1006,WINDSTORM,KISUMU,KISUMU,WINAM,NaN,Luanda Magwar Secondary School,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.073312,34.727767,NYAKACH,"(-0.16945869476, 34.83520346600)"
6,1007,WINDSTORM,KISUMU,KISUMU,WINAM,NaN,Mondi Primary School,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.073312,34.727767,NYAKACH,"(-0.16945869476, 34.83520346600)"
7,1008,DROUGHT,TAITA TAVETA,TAITA TAVETA,WUNDANYI,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-3.284490,38.285085,WUNDANYI,"(-3.43473399370, 38.41860200330)"
8,1009,DROUGHT,TAITA TAVETA,TAITA TAVETA,WUNDANYI,NaN,Wundanyi,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-3.284490,38.285085,WUNDANYI,"(-3.43473399370, 38.41860200330)"
9,1010,LANDSLIDE,TAITA TAVETA,TAITA TAVETA,WUNDANYI,NaN,Mbale location,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-3.284490,38.285085,WUNDANYI,"(-3.43473399370, 38.41860200330)"


In [ ]:
def test_clean(datasets):
    for i in datasets[0:10]:
        print(Dataset.read_from_hdx(i).get_filetypes(),i)
test_clean(datasets)

In [ ]:
dataset_1 = Dataset.read_from_hdx(datasets[datasets.index('160523-ocha-4w-round-2')]) 
dataset_1.separate_resources() 
print(len(dataset_1.resources))
print(dataset_1.get_filetypes()) 
print(dataset_1.resources[0].get_resource_views())

In [ ]:
url, path = dataset_1.resources[0].download('Datasets') #Substitute Datasets with your file download path.
print('Resource URL %s downloaded to %s' % (url, path))

In [ ]:
pandas_dataset = pd.read_csv(path)
pandas_dataset

In [ ]:
pandas_dataset.head(5)